# Analiza tekstu - ciąg dalszy

In [ ]:
import nltk
nltk.download('punkt')

## Preprocessing

### Tokenizacja

In [ ]:
sentences = "The cat is in the box. The cat likes the box. The box is over the cat."

In [ ]:
# tokenizacja
from nltk.tokenize import word_tokenize

tokens = word_tokenize(sentences)
print(tokens)

### Lowercase

In [ ]:
tokens = [token.lower() for token in tokens]
print(tokens)

### Usunięcie tokenów, które nie są alfanumeryczne

In [ ]:
tokens = [token for token in tokens if token.isalpha()]
print(tokens)

### Usunięcie stopwords

**Stopwords** - najczęściej występujące słowa w języku, które nie niosą ze sobą żadnej konkretnej treści.

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

stopwords_list = stopwords.words('english')
print(stopwords_list)

In [ ]:
tokens = [token for token in tokens if token not in stopwords_list]
print(tokens)

### Lematyzacja (alternatywnie: stemming)

In [ ]:
nltk.download('wordnet')

In [ ]:
from nltk.stem import WordNetLemmatizer

# Inicjalizacja WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

tokens = [wordnet_lemmatizer.lemmatize(token) for token in tokens]
print(tokens)

# Techniki osadzania włów (_word embeddings_)

## Model "Bag of Words" (BoW)

In [ ]:
from collections import Counter

c = Counter(tokens)
print(c)

In [ ]:
# dwa najczęściej występujące tokeny w tekście
print(c.most_common(2))

### Biblioteka Gensim

Biblioteka ``Gensim`` jest inną popularną biblioteką do przetwarzania tekstu w Pythonie, która pozwala nam w prosty sposób budować korpusy i słowniki. **Korpus** to zbiór tekstów, na któym wykonujemy zadania przetwarzania języka naturalnego. Składa się z pojedynczych **dokumentów**. Zbiór słów występujących w korpusie nazywamy jego **słownikiem**. Do tokenizacji oraz oczyszczania tekstu użyjemy biblioteki `nltk`.

In [ ]:
from nltk.tokenize import word_tokenize

# nasz dokument (lista napisów)
my_documents = [
    'Nearly all great ideas follow a similar creative process and this article explains how this process works. Understanding this is important because creative thinking is one of the most useful skills you can possess.',
    'Not doing something will always be faster than doing it. This statement reminds me of the old computer programming saying, Remember that there is no code faster than no code.',
    'He went on to become a trailblazer in the field of photography and held over 70 patents by the end of his career. His story of creativity and innovation, which I will share now, is a useful case study for understanding the 5 key steps of the creative process.',
    'He spent the rest of the decade experimenting with new photography techniques and learning about cameras, printers, and optics.',   
]

# preprocessing (lowercase, usuwam znaki interpunkcyjne i stopwords)
tokenized_docs = [word_tokenize(article.lower()) for article in my_documents]

for idx, article_word_list in enumerate(tokenized_docs):
    
    new_article_word_list = []
    for token in article_word_list:
        
        if token.isalpha() and token not in stopwords_list:
            new_article_word_list.append(token)
    
    tokenized_docs[idx] = new_article_word_list

print(tokenized_docs)

Z tak przygotowanego zestawy tokenów możemy stworzyć **słownik** (model _bag of words_), czyli jednoznacznie przyporządkować liczbę każdemu tokenowi:

In [ ]:
#!pip install gensim
from gensim.corpora.dictionary import Dictionary

# Słownik (mapa)
dictionary = Dictionary(tokenized_docs)
print(dictionary.token2id)

Słownik posiada metodę `doc2bow()`, któa zwraca rozkład _bag of words_ przekazanego do niej dokumentu. Zliczane są wyłącznie tokeny występujące w słowniku. Wynikowa lista składa się z dwuelementowych tupli, w któych pierwszy element to id tokena w słowniku, a drugi element to liczba wystąpień tego tokena w przekazanym do metody dokumencie.

In [ ]:
bow = dictionary.doc2bow(tokenized_docs[0])
print(bow)

Następnie z tak przygotowanego słownika możemy otrzymać rozkład  _bag of words_ dla całego **korpusu**:

In [ ]:
# stworzony korpus należy przetworzyć na postać 'DataFrame', aby użyć modeli ML z pakietu 'sklearn'

bow_corpus = [dictionary.doc2bow(article) for article in tokenized_docs]
print(bow_corpus)

**Powyższy korpus** jest trochę czymś innym niż to co przeważnie mamy na myśli mówiąć **korpus języka - czyli zestaw dokumentów**. `Gensim` używa prostego modelu **BoW** za pomocą którego przekształca każdy dokument w **BoW** użwając **id tokenów** i częstość występowania tokenu w dokumencie. Za pomocą `Gensim` w kilku linijkach możemy otrzymać nowy **korpus** i **BoW**.

I ten korpus można łatwo zapisywać, aktualizować i ponownie wykorzystywać dzięki narzędziom biblioteki `Gensim`.

## Model TF-IDF

In [ ]:
print(bow_corpus[0])

In [ ]:
from gensim.models.tfidfmodel import TfidfModel

tfidf_corpus = TfidfModel(bow_corpus)
print(tfidf_corpus[bow_corpus[0]])

Mimo, że _token_ o **id=15** występuje w dokumencie **1 raz** (czyli dokłądnie tyle samo, co tokeny o **id=1, 2, 3, ...**) to wartość przyporządkowana jemu w **tf-idf** jest znacznie niższa. Widocznie ten token występuje częściej w innych dokumentach od pozostałych tokenów pierwszego dokumentu.

In [ ]:
dictionary[15]

## Latent Dirichlet Allocation (LDA)

In [ ]:
from gensim.models.ldamodel import LdaModel

ldamodel = LdaModel(bow_corpus, num_topics=2, id2word = dictionary, passes=20)
print(ldamodel.print_topics(num_topics=2, num_words=3))

## Moduł `scikit-learn`: modele BoW i TD-IDF

In [ ]:
# wersja z pakietu 'scikit-learn'
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# nasz dokument (lista napisów)
my_documents = [
    'Nearly all great ideas follow a similar creative process and this article explains how this process works. Understanding this is important because creative thinking is one of the most useful skills you can possess.',
    'Not doing something will always be faster than doing it. This statement reminds me of the old computer programming saying, Remember that there is no code faster than no code.',
    'He went on to become a trailblazer in the field of photography and held over 70 patents by the end of his career. His story of creativity and innovation, which I will share now, is a useful case study for understanding the 5 key steps of the creative process.',
    'He spent the rest of the decade experimenting with new photography techniques and learning about cameras, printers, and optics.',   
]

# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(my_documents)
v = vectorizer.get_feature_names_out()

print(X.shape)

pd.DataFrame(X.toarray(), columns=v)  # ... i po uprzednim oczyszczeniu danych można trenować model ML